In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly_express as px
import seaborn as sns

palette = sns.color_palette("rainbow", 8)


import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #importing basic visualization libraries
import seaborn as sns  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from mpl_toolkits.mplot3d import Axes3D
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv


In [3]:
#read data
df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', nrows=2_000_000)
test_data = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 122.1+ MB


## 查nan

In [5]:
#df.isnull().sum()

## drop nan

In [6]:
# 删除具有空值的行
df.dropna(subset=['dropoff_longitude', 'dropoff_latitude'], inplace=True)

In [7]:
#df.isnull().sum()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1999986 entries, 0 to 1999999
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 137.3+ MB


## 檢查是否有低於起跳價格2.5美元 https://www.taxifarefinder.com/info.php?city=NY & 乘客>0 <5

In [9]:

df = df[(df['fare_amount'] >= 2.5) & (df['passenger_count'] > 0) & (df['passenger_count'] < 5)]


In [10]:
# 检查 fare_amount 和 passenger_count 列的值是否都大于0
fare_amount_greater_than_zero = (df['fare_amount'] > 0).all()
passenger_count_greater_than_zero = (df['passenger_count'] > 0).all()

if fare_amount_greater_than_zero and passenger_count_greater_than_zero:
    print("All values in fare_amount and passenger_count are greater than 0.")
else:
    print("There are values in fare_amount or passenger_count that are not greater than 0.")


All values in fare_amount and passenger_count are greater than 0.


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1809233 entries, 0 to 1999999
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 124.2+ MB


In [12]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1.809233e+06,1.809233e+06,1.809233e+06,1.809233e+06,1.809233e+06,1.809233e+06
mean,1.135063e+01,-7.252644e+01,3.993446e+01,-7.253039e+01,3.993374e+01,1.331135e+00
std,9.894672e+00,1.290695e+01,8.119578e+00,1.287752e+01,1.057677e+01,6.780948e-01
min,2.500000e+00,-3.377681e+03,-3.458665e+03,-3.383297e+03,-3.461541e+03,1.000000e+00
25%,6.000000e+00,-7.399210e+01,4.073489e+01,-7.399142e+01,4.073400e+01,1.000000e+00
50%,8.500000e+00,-7.398183e+01,4.075259e+01,-7.398018e+01,4.075312e+01,1.000000e+00
75%,1.250000e+01,-7.396718e+01,4.076705e+01,-7.396374e+01,4.076808e+01,1.000000e+00
max,1.273310e+03,2.856442e+03,2.621628e+03,3.414307e+03,3.345917e+03,4.000000e+00


## 時間切割

In [13]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format= "%Y-%m-%d %H:%M:%S UTC")

# 2009-06-15 17:26:21 UTC
# add time information
df['year'] = df.pickup_datetime.apply(lambda t: t.year)
df['month'] = df.pickup_datetime.apply(lambda t: t.month)
df['weekday'] = df.pickup_datetime.apply(lambda t: t.weekday())
df['hour'] = df.pickup_datetime.apply(lambda t: t.hour)



In [14]:
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9


In [15]:
def distance(lat1, lon1, lat2, lon2):
    
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...

In [16]:
df['distance'] = distance(df.pickup_latitude, df.pickup_longitude, 
                                    df.dropoff_latitude, df.dropoff_longitude)

In [17]:
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17,0.640487
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16,5.250670
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0,0.863411
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4,1.739386
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7,1.242218
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13,0.256479
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17,0.615289
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22,2.049851
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9,3.181034


In [18]:
Df=df
Df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17,0.640487
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16,5.250670
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0,0.863411
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4,1.739386
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7,1.242218
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13,0.256479
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17,0.615289
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22,2.049851
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9,3.181034


## Df

In [19]:
Df=Df.drop(['pickup_datetime','key'], axis=1)
Df

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17,0.640487
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16,5.250670
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0,0.863411
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4,1.739386
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7,1.242218
...,...,...,...,...,...,...,...,...,...,...,...
1999995,4.0,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13,0.256479
1999996,7.0,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17,0.615289
1999997,10.5,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22,2.049851
1999998,10.9,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9,3.181034


In [20]:
# Assuming Df is your DataFrame

# Define boundaries based on the provided xlim and ylim
xlim = [-74.03, -73.85]
ylim = [40.70, 40.85]

# Filter data within the specified coordinates
Df = Df[
    (Df['pickup_latitude'].between(ylim[0], ylim[1])) &
    (Df['pickup_longitude'].between(xlim[0], xlim[1])) &
    (Df['dropoff_latitude'].between(ylim[0], ylim[1])) &
    (Df['dropoff_longitude'].between(xlim[0], xlim[1]))
]


In [21]:
zero_distance_rows = Df[Df['distance'] == 0]
zero_distance_rows.count()


fare_amount          15805
pickup_longitude     15805
pickup_latitude      15805
dropoff_longitude    15805
dropoff_latitude     15805
passenger_count      15805
year                 15805
month                15805
weekday              15805
hour                 15805
distance             15805
dtype: int64

In [22]:

# 查看經緯度資料的範圍
print("\nPickup Latitude:")
print(Df['pickup_latitude'].describe())

print("\nPickup Longitude:")
print(Df['pickup_longitude'].describe())

print("\nDropoff Latitude:")
print(Df['dropoff_latitude'].describe())

print("\nDropoff Longitude:")
print(Df['dropoff_longitude'].describe())


Pickup Latitude:
count    1.640759e+06
mean     4.075373e+01
std      2.162023e-02
min      4.070000e+01
25%      4.073897e+01
50%      4.075479e+01
75%      4.076822e+01
max      4.085000e+01
Name: pickup_latitude, dtype: float64

Pickup Longitude:
count    1.640759e+06
mean    -7.397888e+01
std      2.305481e-02
min     -7.402995e+01
25%     -7.399234e+01
50%     -7.398221e+01
75%     -7.396942e+01
max     -7.385009e+01
Name: pickup_longitude, dtype: float64

Dropoff Latitude:
count    1.640759e+06
mean     4.075464e+01
std      2.350127e-02
min      4.070000e+01
25%      4.073921e+01
50%      4.075536e+01
75%      4.076905e+01
max      4.084999e+01
Name: dropoff_latitude, dtype: float64

Dropoff Longitude:
count    1.640759e+06
mean    -7.397735e+01
std      2.400071e-02
min     -7.402999e+01
25%     -7.399173e+01
50%     -7.398108e+01
75%     -7.396726e+01
max     -7.385000e+01
Name: dropoff_longitude, dtype: float64


In [23]:
Df

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16,5.250670
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0,0.863411
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4,1.739386
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7,1.242218
5,12.1,-74.000964,40.731630,-73.972892,40.758233,1,2011,1,3,9,2.353281
...,...,...,...,...,...,...,...,...,...,...,...
1999995,4.0,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13,0.256479
1999996,7.0,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17,0.615289
1999997,10.5,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22,2.049851
1999998,10.9,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9,3.181034


In [24]:
# now removing the key and pickup_datetime coloumn from the dataset
Df = Df.drop(['pickup_longitude', 'pickup_latitude','dropoff_longitude','dropoff_latitude'], axis=1)

In [25]:
Df

,fare_amount,passenger_count,year,month,weekday,hour,distance
1,16.9,1,2010,1,1,16,5.250670
2,5.7,2,2011,8,3,0,0.863411
3,7.7,1,2012,4,5,4,1.739386
4,5.3,1,2010,3,1,7,1.242218
5,12.1,1,2011,1,3,9,2.353281
...,...,...,...,...,...,...,...
1999995,4.0,1,2014,11,0,13,0.256479
1999996,7.0,1,2013,2,0,17,0.615289
1999997,10.5,1,2013,7,4,22,2.049851
1999998,10.9,3,2009,1,5,9,3.181034


In [26]:
# 將特定列轉換為類別型態
columns_to_convert = ['year', 'month', 'weekday', 'hour']

for col in columns_to_convert:
    Df[col] = Df[col].astype('category')

# 查看轉換後的 DataFrame
print(Df.dtypes)

fare_amount         float64
passenger_count       int64
year               category
month              category
weekday            category
hour               category
distance            float64
dtype: object


In [27]:
Df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1640759 entries, 1 to 1999999
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype   
---  ------           --------------    -----   
 0   fare_amount      1640759 non-null  float64 
 1   passenger_count  1640759 non-null  int64   
 2   year             1640759 non-null  category
 3   month            1640759 non-null  category
 4   weekday          1640759 non-null  category
 5   hour             1640759 non-null  category
 6   distance         1640759 non-null  float64 
dtypes: category(4), float64(2), int64(1)
memory usage: 56.3 MB


In [28]:
Df.isnull().sum()

fare_amount        0
passenger_count    0
year               0
month              0
weekday            0
hour               0
distance           0
dtype: int64

In [29]:
#Df=Df.sample(100000)

In [30]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [31]:


# 假设你的 DataFrame 名称为 'df'
# 绘制数据集中所有数值型特征的箱线图
#plt.figure(figsize=(10, 6))  # 可选：设置图形大小
#sns.boxplot(data=Df, orient='h')  # orient='h'表示水平放置箱线图
#plt.title('Boxplot of Numerical Features')  # 设置图标题
#plt.xlabel('Values')  # 设置 x 轴标签
#plt.show()


In [32]:
import seaborn as sns
import matplotlib.pyplot as plt


#plt.figure(figsize=(8, 6))
#sns.boxplot(x='month', y='fare_amount', data=Df)
#plt.title('Boxplot of fare_amount Data by month')
#plt.xlabel('month')
#plt.ylabel('fare_amount')
#plt.show()


In [33]:
import seaborn as sns
import matplotlib.pyplot as plt


#plt.figure(figsize=(8, 6))
#sns.barplot(x='month', y='fare_amount', data=Df)
#plt.title('Barplot of fare_amount Data by month')
#plt.xlabel('month')
#plt.ylabel('fare_amount')
#plt.show()


In [34]:
import seaborn as sns
import matplotlib.pyplot as plt


#plt.figure(figsize=(8, 6))
#sns.barplot(x='weekday', y='fare_amount', data=Df)
#plt.title('Barplot of fare_amount by weekday')
#plt.xlabel('weekday')
#plt.ylabel('fare_amount')
#plt.show()

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt

# 使用 regplot 绘制 fare_amount 和 distance 之间的关系图
#plt.figure(figsize=(8, 6))  # 设置图形大小（可选）
#sns.regplot(x='passenger_count', y='fare_amount', data=Df, scatter_kws={'alpha':0.3})  # 绘制 regplot
#plt.title('Relationship between fare_amount and distance')  # 设置图标题
#plt.xlabel('Distance')  # 设置 x 轴标签
#plt.ylabel('Fare Amount')  # 设置 y 轴标签
#plt.show()


In [36]:
import seaborn as sns
import matplotlib.pyplot as plt

# 创建散点图
#sns.scatterplot(x='distance', y='fare_amount', data=Df)

# 设置图表标题和轴标签
#plt.title('Relationship between Fare Amount and Distance')
#plt.xlabel('Distance')
#plt.ylabel('Fare Amount')
#plt.figure(figsize=(8, 6))  # 设置图形大小（可选）

# 显示图表
#plt.show()


In [37]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# 假設你的 DataFrame 名稱為 df
#columns_to_visualize = ['fare_amount', 'passenger_count', 'year', 'month', 'weekday', 'hour', 'distance']
#mpl.rcParams.update({'font.size': 16})  # 調整字體大小

# 使用 Seaborn 的 pairplot 進行視覺化

#sns.pairplot(Df[columns_to_visualize])

#plt.show()


import seaborn as sns
import matplotlib.pyplot as plt

# 選擇 DataFrame 中的所有數值型態的列
numeric_columns = Df.select_dtypes(include=['float64', 'int64']).columns

# 繪製所有數值型態的列的直方圖
for col in numeric_columns:
    plt.figure(figsize=(8, 6))
    if col == 'distance':
        sns.histplot(data=Df, x=col, bins=20, kde=True, binwidth=0.2)
        plt.xlim(0, 4)  # 調整 x 軸的數值範圍
    else:
        sns.histplot(data=Df, x=col, bins=20, kde=True)
    plt.title(f'Histogram of {col}',fontsize=16)
    plt.xlabel(col,fontsize=16)
    plt.ylabel('Frequency',fontsize=16)
    plt.show()


plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(15, 4))

plt.hist(Df['fare_amount'], bins=100, color='#20beff')
plt.xlabel("fare ($)",fontsize=16)
plt.title("Histrogram plot of fare ($)",fontsize=16)
plt.show()

year_insight = Df.year.value_counts()

plt.figure(figsize=(15, 4))
sns.barplot(data=Df, x=year_insight.index, y=year_insight.values, palette=palette)
plt.ylabel("Number of rides",fontsize=16)
plt.xlabel("Years",fontsize=16)
plt.show()

passenger_count_per_year = Df.groupby('year')['passenger_count'].sum().sort_index()

plt.figure(figsize=(10, 6))
passenger_count_per_year.plot(kind='bar', color='skyblue')
plt.title('Passenger Count per Year', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Total Passenger Count', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()


passenger_count_per_weekday = Df.groupby('weekday')['passenger_count'].sum().sort_index()

# 繪製長條圖
plt.figure(figsize=(8, 6))
passenger_count_per_weekday.plot(kind='bar', color='salmon')
plt.title('Passenger Count per Weekday', fontsize=16)
plt.xlabel('Weekday', fontsize=14)
plt.ylabel('Total Passenger Count', fontsize=14)
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()


fare_per_year = Df.groupby('year')['fare_amount'].mean().sort_index()

# 繪製長條圖
plt.figure(figsize=(8, 6))
fare_per_year.plot(kind='bar', color='skyblue')
plt.title('Average Fare Amount per Year', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Average Fare Amount', fontsize=14)
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()


fare_per_weekday = Df.groupby('weekday')['fare_amount'].mean().sort_index()

# 繪製長條圖
plt.figure(figsize=(8, 6))
fare_per_weekday.plot(kind='bar', color='lightgreen')
plt.title('Average Fare Amount per Weekday', fontsize=16)
plt.xlabel('Weekday', fontsize=14)
plt.ylabel('Average Fare Amount', fontsize=14)
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()


plt.figure(figsize=(8, 6))
plt.scatter(Df['passenger_count'], Df['fare_amount'], alpha=0.5, color='orange')
plt.title('Passenger Count vs Fare Amount', fontsize=16)
plt.xlabel('Passenger Count', fontsize=14)
plt.ylabel('Fare Amount', fontsize=14)
plt.grid(True)
plt.show()


## 從'訓練集'切分並預測

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 假設資料存在名為 df 的 DataFrame 中

# 提取特徵和目標值
X = Df.drop(columns=['fare_amount'])
y = Df['fare_amount']

# 分割資料為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立並訓練線性回歸模型
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# 預測測試集
linear_predictions = linear_model.predict(X_test)

# 計算線性回歸模型的均方誤差
linear_mse = mean_squared_error(y_test, linear_predictions)
print(f"線性回歸模型的均方誤差 (MSE): {linear_mse}")
# 假設 linear_mse 是先前計算得到的均方誤差
linear_rmse = mean_squared_error(y_test, linear_predictions, squared=False)
print(f"線性回歸模型的均方根誤差 (RMSE): {linear_rmse}")

線性回歸模型的均方誤差 (MSE): 14.26815340010842
線性回歸模型的均方根誤差 (RMSE): 3.7773209289267995


In [39]:
from sklearn.metrics import accuracy_score

# 轉換預測值為二元分類
threshold = 10  # 可以根據情況調整閾值
binary_predictions = (linear_predictions >= threshold).astype(int)

# 計算準確率
accuracy = accuracy_score((y_test >= threshold).astype(int), binary_predictions)
print(f"分類準確率 (Accuracy): {accuracy}")

分類準確率 (Accuracy): 0.8675674687340014


In [40]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# 假設資料存在名為 df 的 DataFrame 中

# 要編碼的欄位名稱
categorical_columns = ['year', 'month', 'weekday', 'hour']

# 使用 LabelEncoder 將類別型資料轉換為數值型
label_encoder = LabelEncoder()
for col in categorical_columns:
    Df[col] = label_encoder.fit_transform(Df[col])

# 接下來建立 XGBoost 模型和訓練，使用前面提到的程式碼

# 假設資料存在名為 df 的 DataFrame 中

# 提取特徵和目標值
X = Df.drop(columns=['fare_amount'])
y = Df['fare_amount']

# 分割資料為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立XGBoost回歸模型
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100)

# 訓練XGBoost模型
xgb_model.fit(X_train, y_train)

# 預測測試集
xgb_predictions = xgb_model.predict(X_test)

# 計算XGBoost模型的均方誤差
xgb_mse = mean_squared_error(y_test, xgb_predictions)
print(f"XGBoost模型的均方誤差 (MSE): {xgb_mse}")
xgb_rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
print(f"XGB模型的均方根誤差 (RMSE): {xgb_rmse}")

XGBoost模型的均方誤差 (MSE): 15.157811157338287
XGB模型的均方根誤差 (RMSE): 3.8933033734013445


In [41]:

# 設定閾值來轉換成二元分類
threshold = 10  # 可以根據實際情況調整閾值
binary_predictions = (xgb_predictions >= threshold).astype(int)

# 轉換測試集的目標值為二元分類
binary_actual = (y_test >= threshold).astype(int)

# 計算準確率
accuracy = accuracy_score(binary_actual, binary_predictions)
print(f"準確率 (Accuracy): {accuracy}")

準確率 (Accuracy): 0.8710871791121187


## 從'測試集'預測

In [42]:
# 將 pickup_datetime 轉換為 datetime 格式
test_data['pickup_datetime'] = pd.to_datetime(test_data['pickup_datetime'])

# 提取年、月、星期幾、幾點資訊
test_data['year'] = test_data['pickup_datetime'].dt.year.astype('category')
test_data['month'] = test_data['pickup_datetime'].dt.month.astype('category')
test_data['weekday'] = test_data['pickup_datetime'].dt.dayofweek.astype('category')
test_data['hour'] = test_data['pickup_datetime'].dt.hour.astype('category')
test_data = test_data.drop(['pickup_datetime'], axis=1)

In [43]:
test_data

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour
0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,1,13
1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,1,13
2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,5,11
3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,5,21
4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,5,21
...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,-73.968124,40.796997,-73.955643,40.780388,6,2015,5,6,12
9910,2015-01-12 17:05:51.0000001,-73.945511,40.803600,-73.960213,40.776371,6,2015,1,0,17
9911,2015-04-19 20:44:15.0000001,-73.991600,40.726608,-73.789742,40.647011,6,2015,4,6,20
9912,2015-01-31 01:05:19.0000005,-73.985573,40.735432,-73.939178,40.801731,6,2015,1,5,1


In [44]:
Df

,fare_amount,passenger_count,year,month,weekday,hour,distance
1,16.9,1,1,0,1,16,5.250670
2,5.7,2,2,7,3,0,0.863411
3,7.7,1,3,3,5,4,1.739386
4,5.3,1,1,2,1,7,1.242218
5,12.1,1,2,0,3,9,2.353281
...,...,...,...,...,...,...,...
1999995,4.0,1,5,10,0,13,0.256479
1999996,7.0,1,4,1,0,17,0.615289
1999997,10.5,1,4,6,4,22,2.049851
1999998,10.9,3,0,0,5,9,3.181034


In [45]:
test_data['distance'] = distance(test_data.pickup_latitude, test_data.pickup_longitude, 
                                    test_data.dropoff_latitude, test_data.dropoff_longitude)
test_data = test_data.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'], axis=1)
test_data

,key,passenger_count,year,month,weekday,hour,distance
0,2015-01-27 13:08:24.0000002,1,2015,1,1,13,1.443607
1,2015-01-27 13:08:24.0000003,1,2015,1,1,13,1.507044
2,2011-10-08 11:53:44.0000002,1,2011,10,5,11,0.384398
3,2012-12-01 21:12:12.0000002,1,2012,12,5,21,1.218529
4,2012-12-01 21:12:12.0000003,1,2012,12,5,21,3.347514
...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,6,2015,5,6,12,1.320336
9910,2015-01-12 17:05:51.0000001,6,2015,1,0,17,2.032486
9911,2015-04-19 20:44:15.0000001,6,2015,4,6,20,11.920349
9912,2015-01-31 01:05:19.0000005,6,2015,1,5,1,5.184402


In [46]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [47]:
# 假設你有一個名為 df 的 DataFrame，其中包含了訓練資料集

# 假設 df 包含目標值在名為 'target' 的欄位中，並且其他欄位是特徵
# 如果需要，可以分割資料為訓練集和測試集
# train_df, test_df = train_test_split(df, test_size=0.2)

# 從訓練資料集中提取特徵和目標
# 這裡假設 'target' 是目標列
X_train = Df.drop(columns=['fare_amount'])
y_train = Df['fare_amount']

# 如果有必要，對特徵進行標準化或其他前處理
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

# 建立並訓練線性回歸模型
model = LinearRegression()
model.fit(X_train, y_train)



# 提取測試集特徵資料
test_features = test_data[['passenger_count','year','month','weekday','hour','distance']]

# 使用模型進行預測
predictions = model.predict(test_features)

# 將預測結果加入測試集資料中（如果需要）
test_data['predicted_fare'] = predictions

# 儲存包含預測結果的測試集資料（如果需要）
test_data.to_csv('predicted_test_data_linear_1116.csv', index=False)  # 儲存到一個新的 CSV 檔案中

In [48]:
from sklearn.model_selection import cross_val_score

# 計算 R² score
r2 = model.score(X_train, y_train)
print(f"R² score: {r2}")

# 使用 K-fold 交叉驗證計算 R² score
num_folds = 5  # 定義交叉驗證的 fold 數量
cv_r2_scores = cross_val_score(model, X_train, y_train, cv=num_folds, scoring='r2')

print(f"Cross-validated R² scores: {cv_r2_scores}")
print(f"Mean R² score: {np.mean(cv_r2_scores)}")


R² score: 0.6567154728837994
Cross-validated R² scores: [0.63841941 0.67051652 0.65431906 0.65776838 0.66356245]
Mean R² score: 0.6569171632292463


In [49]:
from sklearn.preprocessing import LabelEncoder

# 從訓練資料集中提取特徵和目標
# 這裡假設 'target' 是目標列
X_train = Df.drop(columns=['fare_amount','year','month','weekday','hour'])
y_train = Df['fare_amount']
# 建立XGBoost回歸模型
model = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                          max_depth=5, alpha=10, n_estimators=100)

# 訓練模型
model.fit(X_train, y_train)

# 使用模型進行預測
predictions = model.predict(test_features[['passenger_count','distance']])

# 將預測結果加入測試集資料中
test_data['predicted_fare'] = predictions
submission=test_data[['key','predicted_fare']]
# 儲存包含預測結果的測試集資料
submission.to_csv('submission.csv', index=False)


In [50]:
from sklearn.model_selection import cross_val_score

# 計算 R² score
r2 = model.score(X_train, y_train)
print(f"R² score: {r2}")

# 使用 K-fold 交叉驗證計算 R² score
num_folds = 5  # 定義交叉驗證的 fold 數量
cv_r2_scores = cross_val_score(model, X_train, y_train, cv=num_folds, scoring='r2')

print(f"Cross-validated R² scores: {cv_r2_scores}")
print(f"Mean R² score: {np.mean(cv_r2_scores)}")


R² score: 0.6647460461996817
Cross-validated R² scores: [0.64674132 0.67739996 0.66170235 0.66721179 0.67124496]
Mean R² score: 0.6648600763825006


In [51]:
submission

,key,predicted_fare
0,2015-01-27 13:08:24.0000002,8.819118
1,2015-01-27 13:08:24.0000003,9.076632
2,2011-10-08 11:53:44.0000002,4.814044
3,2012-12-01 21:12:12.0000002,8.014013
4,2012-12-01 21:12:12.0000003,15.173026
...,...,...
9909,2015-05-10 12:37:51.0000002,8.601445
9910,2015-01-12 17:05:51.0000001,11.120235
9911,2015-04-19 20:44:15.0000001,34.750744
9912,2015-01-31 01:05:19.0000005,22.927328
